In [1]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'data/train_subset.csv', 'eval': 'data/eval_subset.csv'})

In [2]:
from utils import transform_labels
dataset = dataset.map(transform_labels)

In [3]:
# let's train a Distilbert model

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')

# let's tokenize the data for the model to be able to understand
def tokenize_data(example):
    return tokenizer(example['Sentence'], padding='max_length')    

In [4]:
dataset = dataset.map(tokenize_data, batched=True)

In [5]:
from transformers import AutoModelForSequenceClassification

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", num_labels=3)

In [6]:
# the default batch size for training arguments
batch_size = 8

# set number of epochs
number_of_epochs = 7
# let set the logging steps
logging_steps = len(dataset['train']) // batch_size # it should log each batch 

steps = (len(dataset['train']) / batch_size) * number_of_epochs
warmup_steps = int(0.2 * steps)

In [7]:
from transformers import TrainingArguments
training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs, 
                                  load_best_model_at_end=True,
                                  evaluation_strategy='steps', 
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir="fine-tuned-distilbert-base-uncased"
                                  )

e:\repo\DistilBERTFinancialSentiment\.venv\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['eval'].shuffle(seed=10)

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [10]:
# Launch the learning process: training
# Load model from checkpoint
trainer.train()

Step,Training Loss,Validation Loss
500,1.308500,0.840402
1000,0.839500,0.862964
1500,0.719200,1.040590
2000,0.452300,1.002797
2500,0.360600,1.349325


TrainOutput(global_step=2800, training_loss=0.65254668372018, metrics={'train_runtime': 345.8809, 'train_samples_per_second': 64.762, 'train_steps_per_second': 8.095, 'total_flos': 2967322646937600.0, 'train_loss': 0.65254668372018, 'epoch': 7.0})

In [11]:
from utils import compute_metrics

trainer_eval = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [12]:
trainer_eval.evaluate()

Trainer is attempting to log a value of "{'accuracy': 0.6775}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.6747647004734894}" of type <class 'dict'> for key "eval/f1score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 1.2857215404510498,
 'eval_model_preparation_time': 0.0032,
 'eval_accuracy': {'accuracy': 0.6775},
 'eval_f1score': {'f1': 0.6747647004734894},
 'eval_runtime': 6.1421,
 'eval_samples_per_second': 130.249,
 'eval_steps_per_second': 16.281}